# V6: fract2

In [35]:
from parase_input_package.generate_output import *
from parase_input_package.parase_input import *
from parase_input_package.plot_output import *
filepath_out='../output/fract2.router'
netlist_file_path = '../benchmark/fract2.nl'
gridfile_path  = '../benchmark/fract2.grid'
nets,net_num = parse_netlist(netlist_file_path)
rows,columns,bend_penalty,via_penalty,layer1_grid_original,layer2_grid_original = parse_gridfile(gridfile_path)

bend_penalty: 10 
 via_penalty: 20
size of layer1: (127, 231)
size of layer2: (127, 231)


In [36]:
# plot problem 根据grid文件可视化障碍物和每个格子的cost，同时标出source和target，可以显示两层
plot_problem("../output/fract2_problem.jpg",columns,rows,layer1_grid_original,layer2_grid_original,nets)

In [37]:
def reconstruct_path(source, target, parents):
    path = []
    current = target
    while current != source:
        path.append(current)
        current = parents[current]
    path.append(source)
    path.reverse()
    return path

def reconstruct_path_tmp(source, target_tmp, parents):
    path_tmp = []
    current = target_tmp
    while current != source and len(path_tmp)<= 3:
        path_tmp.append(current)
        current = parents[current]
    path_tmp.append(source)
    path_tmp.reverse()
    return path_tmp

def mark_path_on_grid(layer1_grid,layer2_grid, path):
    if path:               
        for cell in path:
            x, y, layer = cell
            if  (layer == 1):
                layer1_grid[x][y] = -1
            if  (layer == 2):
                layer2_grid[x][y] = -1

def get_cell_cost(layer_grid, cell,path_tmp,bend_penalty,via_penalty):  
    x, y, layer = cell
    cell_cost = 1  # 默认的单元代价
    if layer_grid[x][y] == -1:
        cell_cost = float('inf')  # -1表示无法通过的细胞
    elif layer_grid[x][y] != 1:
        cell_cost = layer_grid[x][y]  # 非单元代价
    if len(path_tmp) >= 2:
        prev_cell = path_tmp[-2]
        prev_x, prev_y, prev_layer = prev_cell
        if layer != prev_layer:
            cell_cost += via_penalty
        elif (prev_x != x and prev_y != y):  
            cell_cost += bend_penalty
    return cell_cost

def expand_source_to_target(rows, columns, layer1_grid,layer2_grid, source, target,bend_penalty,via_penalty):
    wavefront = {}
    visited = set()
    parents = {}
    costs = {}  # Store the cumulative costs for each cell
    costs_targe = {} 
    
    
    source_tuple = (source['x'], source['y'], source['layer'])
    target_tuple = (target['x'], target['y'], target['layer'])
    
    wavefront[source_tuple] = 0
    costs[source_tuple] = 1  # Initial cost for the source cell is 1

    while wavefront:
        # get lowest cost cell on a wavefront structure
        current_cell = sorted(wavefront.items(),key=lambda s:int(s[1]))[0][0]

        if current_cell == target_tuple:
            path = reconstruct_path(source_tuple, target_tuple, parents)
            return path,costs[current_cell]

        neighbors = get_neighbors(rows, columns, current_cell)

        for neighbor in neighbors:
            neighbor_tuple = (neighbor['x'], neighbor['y'], neighbor['layer'])
            if neighbor_tuple not in visited:
                path_tmp= reconstruct_path_tmp(source_tuple, current_cell, parents)
                # Calculate the cost to reach the neighbor cell
                if(neighbor_tuple[2]== 1):
                  cost = costs[current_cell] + get_cell_cost(layer1_grid, neighbor_tuple,path_tmp,bend_penalty,via_penalty)
                if(neighbor_tuple[2]== 2):
                  cost = costs[current_cell] + get_cell_cost(layer2_grid, neighbor_tuple,path_tmp,bend_penalty,via_penalty)
                # ignore blocks
                if cost!= np.inf:
                    if neighbor_tuple not in wavefront.keys() or costs[neighbor_tuple] > cost:
                        costs[neighbor_tuple] = cost
                        parents[neighbor_tuple] = current_cell
                    if neighbor_tuple not in wavefront.keys():
                        # add cell N to waveform, indexed by pathcost
                        costs_targe = cost + abs(neighbor_tuple[0] - target_tuple[0] ) + abs(neighbor_tuple[1] - target_tuple[1] ) 
                        wavefront[neighbor_tuple]=costs_targe         

        visited.add(current_cell)    
        del wavefront[current_cell]                  
    return None,None

def get_neighbors(rows, columns, cell):  
    x, y, layer = cell
    neighbors = []
    if layer == 1: 
        if x < columns - 1:
            neighbors.append({'x': x + 1, 'y': y, 'layer': layer})
        if x > 0:
            neighbors.append({'x': x - 1, 'y': y, 'layer': layer})  
        neighbors.append({'x': x, 'y': y, 'layer': 3-layer})    
    elif layer == 2: 
        if y < rows - 1:
            neighbors.append({'x': x, 'y': y + 1, 'layer': layer})
        if y > 0:
            neighbors.append({'x': x, 'y': y - 1, 'layer': layer})  
        neighbors.append({'x': x, 'y': y, 'layer': 3-layer})                                       
    return neighbors

def true_two_layer_router(rows, columns, layer1_grid,layer2_grid, nets,bend_penalty,via_penalty):
    nets.sort(key=lambda s: (abs(s["pin1"]['x']-s["pin2"]['x'])+abs(s["pin1"]['y']-s["pin2"]['y'])),reverse = False)
    routing_table = {}
    costs_table = {}
    for net in nets:
        net_id = net['net_id']
        pin1 = net['pin1']
        pin2 = net['pin2']
        # 防止布线在后续的pin上，先将所有的pin标记为-1；
        if pin1['layer'] == 1:
            layer1_grid[pin1['x']][pin1['y']] = -1
        if pin1['layer'] == 2:
            layer2_grid[pin1['x']][pin1['y']] = -1
        if pin2['layer'] == 1:
            layer1_grid[pin2['x']][pin2['y']] = -1
        if pin2['layer'] == 2:
            layer2_grid[pin2['x']][pin2['y']] = -1
           
    for net in nets:
        net_id = net['net_id']
        pin1 = net['pin1']
        pin2 = net['pin2']
        if pin1['layer'] == 1:
            layer1_grid[pin1['x']][pin1['y']] = 1
        if pin1['layer'] == 2:
            layer2_grid[pin1['x']][pin1['y']] = 1
        if pin2['layer'] == 1:
            layer1_grid[pin2['x']][pin2['y']] = 1
        if pin2['layer'] == 2:
            layer2_grid[pin2['x']][pin2['y']] = 1
        print('Routing net:',net_id)
        path,costs = expand_source_to_target(rows, columns, layer1_grid,layer2_grid, pin1, pin2,bend_penalty,via_penalty) 
        if path is not None:
            mark_path_on_grid(layer1_grid,layer2_grid,path)
            routing_table[net_id] = path
            costs_table[net_id] = costs
        elif path is None:
            routing_table[net_id] = None
            costs_table[net_id] = None
    routing_table = dict(sorted(routing_table.items(),key=lambda x:x[0]))
    costs_table = dict(sorted(costs_table.items(),key=lambda x:x[0]))
    return routing_table,costs_table

In [38]:
layer1_grid = layer1_grid_original.copy()
layer2_grid = layer2_grid_original.copy()
routing_table,costs_table=true_two_layer_router(rows, columns, layer1_grid.T,layer2_grid.T, nets,bend_penalty,via_penalty)
plot_path('../output/fract2.jpg',columns=columns,rows=rows,grid1=layer1_grid_original,grid2=layer2_grid_original,path_dict=routing_table) 
generate_output_file(filepath_out,net_num,routing_table)

Routing net: 95
Routing net: 109
Routing net: 119
Routing net: 64
Routing net: 74
Routing net: 105
Routing net: 14
Routing net: 20
Routing net: 22
Routing net: 40
Routing net: 84
Routing net: 12
Routing net: 38
Routing net: 80
Routing net: 42
Routing net: 51
Routing net: 79
Routing net: 103
Routing net: 9
Routing net: 11
Routing net: 37
Routing net: 53
Routing net: 91
Routing net: 33
Routing net: 67
Routing net: 117
Routing net: 13
Routing net: 94
Routing net: 101
Routing net: 116
Routing net: 118
Routing net: 102
Routing net: 4
Routing net: 123
Routing net: 78
Routing net: 111
Routing net: 113
Routing net: 115
Routing net: 49
Routing net: 112
Routing net: 114
Routing net: 26
Routing net: 92
Routing net: 107
Routing net: 108
Routing net: 10
Routing net: 81
Routing net: 54
Routing net: 63
Routing net: 120
Routing net: 31
Routing net: 45
Routing net: 76
Routing net: 1
Routing net: 2
Routing net: 100
Routing net: 110
Routing net: 50
Routing net: 106
Routing net: 44
Routing net: 72
Routing

In [39]:
costs_table


{1: 179,
 2: 179,
 3: 305,
 4: 169,
 5: 312,
 6: 193,
 7: 202,
 8: 197,
 9: 82,
 10: 176,
 11: 168,
 12: 160,
 13: 104,
 14: 154,
 15: 207,
 16: 288,
 17: 227,
 18: 196,
 19: 284,
 20: 154,
 21: 214,
 22: 154,
 23: 299,
 24: 190,
 25: 186,
 26: 110,
 27: 185,
 28: 208,
 29: 202,
 30: 217,
 31: 176,
 32: 197,
 33: 103,
 34: 186,
 35: 227,
 36: 203,
 37: 168,
 38: 160,
 39: 188,
 40: 154,
 41: 222,
 42: 165,
 43: 244,
 44: 182,
 45: 178,
 46: 185,
 47: 195,
 48: 167,
 49: 173,
 50: 181,
 51: 61,
 52: 103,
 53: 82,
 54: 177,
 55: 305,
 56: 257,
 57: 229,
 58: 271,
 59: 207,
 60: 269,
 61: 181,
 62: 140,
 63: 177,
 64: 6,
 65: 189,
 66: 268,
 67: 103,
 68: 197,
 69: 193,
 70: 214,
 71: 201,
 72: 186,
 73: 165,
 74: 6,
 75: 113,
 76: 176,
 77: 196,
 78: 170,
 79: 61,
 80: 161,
 81: 174,
 82: 216,
 83: 262,
 84: 156,
 85: 182,
 86: 215,
 87: 173,
 88: 194,
 89: 124,
 90: 316,
 91: 82,
 92: 174,
 93: 236,
 94: 84,
 95: 4,
 96: 189,
 97: 223,
 98: 193,
 99: 212,
 100: 177,
 101: 166,
 102: 167

In [40]:
sum(costs_table.values()) 
#单一方向：由难到易15372，完成124
#四个方向，由易到难12310，完成99
#单一方向，由易到难21982，完成125

21982

In [41]:
cnt = 0
for item in routing_table.keys():
    if routing_table[item]:
        cnt += 1
cnt

125

In [1]:
 from parase_input_package.evaluate import *
 router_path = '../output/fract2.router'
 bench_name ="fract2"
 duplicate_points ,cost_path,total_cost=evaluate_route(router_path,bench_name)

所有的节点均已布线.
未找到重复的点。
所有布线都连续
bend_penalty: 10 
 via_penalty: 20
布线结果的起点和终点与Nets全部匹配
total_cost: 15222.0
